In [1]:
!pip install httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


# Compare https with requests

Reference : https://youtu.be/qAh5dDODJ5k

[basic methods]

In [1]:
import httpx
import requests
import time

In [2]:
def main_requests():
  pokemons = []
  for number in range(1,151):
    pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{number}'
    res = requests.get(pokemon_url)
    pokemons.append(res.json()['name'])

In [3]:
start_time = time.time()
main_requests()
print(f"Requests: {time.time() - start_time :.2f} seconds.")

Requests: 18.56 seconds.


In [4]:
def main_httpx():
  pokemons = []
  for number in range(1,151):
    pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{number}'
    res = httpx.get(pokemon_url)
    pokemons.append(res.json()['name'])

In [5]:
start_time = time.time()
main_httpx()
print(f"HTTPX: {time.time() - start_time :.2f} seconds.")

HTTPX: 17.76 seconds.


[Use session - client (httpx)]

In [6]:
def main_requests():
  pokemons = []
  with requests.Session() as sess:

    for number in range(1,151):
      pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{number}'
      res = sess.get(pokemon_url)
      pokemons.append(res.json()['name'])

In [7]:
start_time = time.time()
main_requests()
print(f"Requests with Session(): {time.time() - start_time :.2f} seconds.")

Requests with Session(): 4.53 seconds.


In [8]:
def main_httpx():
  pokemons = []
  with httpx.Client() as client:
    for number in range(1,151):
      pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{number}'
      res = client.get(pokemon_url)
      pokemons.append(res.json()['name'])

In [9]:
start_time = time.time()
main_httpx()
print(f"HTTPX with Client(): {time.time() - start_time :.2f} seconds.")

HTTPX with Client(): 4.53 seconds.


- No improvement

[Use async in wrong way]

- In the reference, the method below is a wrong way, but it is confirmed that the speed is improved.

In [10]:
import asyncio

async def main1():
  async with httpx.AsyncClient() as client:
    pokemons = []
    for number in range(1,151):
      pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{number}'
      res = await client.get(pokemon_url)
      pokemons.append(res.json()['name'])

In [11]:
start_time = time.time()
await main1()
print(f"HTTPX - asnyc main_1 with Client(): {time.time() - start_time :.2f} seconds.")

HTTPX - asnyc main_1 with Client(): 4.50 seconds.


[better way]

In [12]:
async def get_pokemon(client, url):
  res = await client.get(url)
  return res.json()['name']

In [13]:
async def main2():
  async with httpx.AsyncClient() as client:
    tasks = []
    for number in range(1,151):
      pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{number}'
      tasks.append(asyncio.create_task(get_pokemon(client,pokemon_url)))
    original_pokemon = await asyncio.gather(*tasks)

In [14]:
start_time = time.time()
await main2()
print(f"HTTPX - asnyc main_2 with Client(): {time.time() - start_time :.2f} seconds.")

HTTPX - asnyc main_2 with Client(): 2.02 seconds.


👍